# prepocessing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
#cleaning text data
import nltk
import numpy as np
import re #regular expressions(rmeoving fullstops etc)
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91738\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataset= pd.read_csv('train.csv',encoding = "ISO-8859-1",nrows=9000)
dataset

,sentiment,text
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...
...,...,...
8995,0,&quot;#WikiLeaks is overloaded by global inter...
8996,0,"&quot;&lt; monica&gt; tenk pÃÂ¥ Neophos, da &..."
8997,0,&quot;&quot;Doing some work&quot;&quot; Facebo...
8998,1,&quot;&quot;I'm really glad you're in my life....


# removing punctuation and number

In [3]:
def preprocess_tweet(tweet):
    #Preprocess the text in a single tweet
    #arguments: tweet = a single tweet in form of string 
    #convert the tweet to lower case
    tweet.lower()
    #convert all urls to sting "URL"
    tweet= re.sub('[^a-zA-Z]',' ',tweet)
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #convert all @username to "AT_USER"
    tweet = re.sub('@[^\s]+','AT_USER', tweet)
    #correct all multiple white spaces to a single white space
    tweet = re.sub('[\s]+', ' ', tweet)
    #convert "#topic" to just "topic"
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    return tweet
dataset['text'] = dataset['text'].apply(preprocess_tweet)

# Stemming

In [4]:
c=[]
for i in range(0,9000):
    review1= re.sub('[^a-zA-Z]',' ',dataset['text'].iloc[i])
    #cover it as lower case
    review1=review1.split()#splits words and forms as list
    ps= PorterStemmer()
    review1=[ps.stem(word) for word in review1 if not word in set(stopwords.words('english'))]
    review1=' '.join(review1)
    c.append(review1)

In [5]:
#creating a bag of words model

from  sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=15000)
x= cv.fit_transform(c).toarray()
y= dataset.iloc[:,0:1].values

# splitting into training and testing data

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)
pickle.dump(cv.vocabulary_,open("cf19.pkl","wb"))

# importing libraries for the model

In [7]:
from keras.models import Sequential
from keras.layers import Dense
model=Sequential()

Using TensorFlow backend.
C:\Users\91738\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\91738\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\91738\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\91738\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

# Adding Input layer

In [8]:
model.add(Dense(input_dim=x.shape[1],init="random_uniform",activation="sigmoid",output_dim=50))


C:\Users\91738\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=12923, activation="sigmoid", units=50, kernel_initializer="random_uniform")`
  """Entry point for launching an IPython kernel.


# Adding hidden layer

In [9]:
model.add(Dense(init="random_uniform",activation="sigmoid",output_dim=10))


C:\Users\91738\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=10, kernel_initializer="random_uniform")`
  """Entry point for launching an IPython kernel.


# Adding output layer

In [10]:
model.add(Dense(output_dim=1,init='random_uniform',activation='sigmoid'))


C:\Users\91738\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="random_uniform")`
  """Entry point for launching an IPython kernel.


# configure the learning process and optimization

In [11]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
#optimizing model

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# training model

In [12]:
model.fit(x_train,y_train,epochs=50,batch_size=130)


Epoch 1/50
6300/6300 [==============================] - 1s 212us/step - loss: 0.6746 - accuracy: 0.6048
Epoch 2/50
6300/6300 [==============================] - 1s 161us/step - loss: 0.6698 - accuracy: 0.6048
Epoch 3/50
6300/6300 [==============================] - 1s 168us/step - loss: 0.6685 - accuracy: 0.6048
Epoch 4/50
6300/6300 [==============================] - 1s 164us/step - loss: 0.6663 - accuracy: 0.6048
Epoch 5/50
6300/6300 [==============================] - 1s 160us/step - loss: 0.6635 - accuracy: 0.6048
Epoch 6/50
6300/6300 [==============================] - 1s 189us/step - loss: 0.6584 - accuracy: 0.6048
Epoch 7/50
6300/6300 [==============================] - 1s 203us/step - loss: 0.6486 - accuracy: 0.6048
Epoch 8/50
6300/6300 [==============================] - 1s 197us/step - loss: 0.6290 - accuracy: 0.6048
Epoch 9/50
6300/6300 [==============================] - 1s 168us/step - loss: 0.5949 - accuracy: 0.6198
Epoch 10/50
6300/6300 [==============================] - 1s 172

# Testing

In [13]:
y_pred=model.predict(x_test)
y_pred=(y_pred>0.5)

# save the model

In [14]:
model.save('final3.h5')

# confusion matrix for accuracy

In [15]:
#confusion matrix
#accuracy
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)
loaded_vec =CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("cf19.pkl", "rb")))
da=""
da=da.split("delimiter")
result= model.predict(loaded_vec.transform(da))


[[1283  399]
 [ 347  671]]


# predictions

In [16]:
r=model.predict(cv.transform(["@babymakinmachin thank for the shout out!   Did you ever find a good flight?"]))
if r>0.5:
    print("The expression has positive expression")
else:
    print("The expression has negative expression")

The expression has positive expression
